# Fraud-detection PoC: DuckDB + Embeddings + Retrieval + (Mock/Optional) LLM

This notebook demonstrates a runnable end-to-end proof-of-concept using DuckDB for data management in a fraud-detection workflow. It covers:
- Ingesting synthetic transactions into DuckDB
- Computing text embeddings for transaction descriptions
- Performing nearest-neighbour retrieval (semantic similarity)
- Calling an LLM (mocked if no API key) to get an explanation and risk score
- Storing LLM outputs and provenance back into DuckDB

Notes:
- This is a PoC for demonstration and data-management patterns; adapt storage, PII masking, and production vector stores (FAISS/Milvus) for production.
- If you want real LLM calls, set the environment variable `OPENAI_API_KEY` before running and the notebook will call OpenAI's chat completion API; otherwise a deterministic mock LLM will be used.


In [ ]:
### Install dependencies (run once)
# Uncomment the next line if you need to install packages in the environment running this notebook.
# Note: model download (sentence-transformers) occurs in the embeddings cell and may take time.

# !pip install -q duckdb sentence-transformers scikit-learn pandas numpy openai
pass

In [ ]:
import os
import time
import json
import hashlib
from datetime import datetime
import duckdb
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sentence_transformers import SentenceTransformer
try:
    import openai
except Exception:
    openai = None

print('libraries loaded')

# Config
DB_FILE = ':memory:'  # for demo; replace with 'fraud_poc.duckdb' to persist to disk
EMB_MODEL_NAME = 'all-MiniLM-L6-v2'  # small, fast SBERT model
TOP_K = 5
OPENAI_KEY = os.getenv('OPENAI_API_KEY')
if OPENAI_KEY and openai:
    openai.api_key = OPENAI_KEY
    REAL_LLM_AVAILABLE = True
else:
    REAL_LLM_AVAILABLE = False

print('REAL_LLM_AVAILABLE =', REAL_LLM_AVAILABLE)


In [ ]:
# Create DuckDB connection and tables
con = duckdb.connect(DB_FILE)

con.execute('''
CREATE TABLE IF NOT EXISTS transactions (
  tx_id VARCHAR PRIMARY KEY,
  account_id VARCHAR,
  amount DOUBLE,
  currency VARCHAR,
  merchant VARCHAR,
  merchant_mcc VARCHAR,
  timestamp TIMESTAMP,
  description VARCHAR,
  device_info VARCHAR,
  geo_country VARCHAR,
  ingestion_job_id VARCHAR,
  raw_source VARCHAR,
  pii_masked BOOLEAN DEFAULT FALSE,
  created_at TIMESTAMP DEFAULT current_timestamp
);
''')

con.execute('''
CREATE TABLE IF NOT EXISTS embeddings (
  tx_id VARCHAR PRIMARY KEY,
  emb_json VARCHAR,
  emb_model VARCHAR,
  emb_created_at TIMESTAMP DEFAULT current_timestamp,
  emb_job_id VARCHAR
);
''')

con.execute('''
CREATE TABLE IF NOT EXISTS llm_results (
  id VARCHAR PRIMARY KEY,
  tx_id VARCHAR,
  llm_model VARCHAR,
  llm_provider VARCHAR,
  llm_prompt_hash VARCHAR,
  llm_prompt VARCHAR,
  llm_response VARCHAR,
  parsed_response JSON,
  risk_score DOUBLE,
  evidence_tx_ids VARCHAR,
  call_latency_ms INTEGER,
  usage JSON,
  created_at TIMESTAMP DEFAULT current_timestamp
);
''')

print('DuckDB tables created')


In [ ]:
# Ingest some synthetic transactions (including labeled fraud examples)
ingestion_job_id = hashlib.sha1(str(time.time()).encode()).hexdigest()[:10]
sample = [
    ("tx1001","acct1", 120.0, "USD", "MerchantA","5812","2025-12-01 10:00:00","POS purchase at MerchantA","chrome","US", ingestion_job_id, 'synthetic', False),
    ("tx1002","acct2", 9500.0, "USD", "MerchantB","6011","2025-12-02 11:00:00","Wire transfer to offshore account","mobile","CN", ingestion_job_id, 'synthetic', False),
    ("tx1003","acct3", 45.0, "USD", "MerchantC","5999","2025-12-03 09:00:00","Online subscription monthly","chrome","US", ingestion_job_id, 'synthetic', False),
    ("tx1004","acct4", 200.0, "USD", "MerchantD","5411","2025-12-04 12:30:00","Large POS purchase unusual location","safari","FR", ingestion_job_id, 'synthetic', False),
    ("tx1005","acct2", 8800.0, "USD", "MerchantE","6011","2025-12-05 15:10:00","Transfer to newly added beneficiary","mobile","CN", ingestion_job_id, 'synthetic', False)
]

df = pd.DataFrame(sample, columns=["tx_id","account_id","amount","currency","merchant","merchant_mcc","timestamp","description","device_info","geo_country","ingestion_job_id","raw_source","pii_masked"])
con.execute("INSERT INTO transactions SELECT * FROM df", {'df': df})
print('Inserted synthetic transactions:')
print(con.execute('SELECT tx_id, amount, description FROM transactions ORDER BY tx_id').fetchdf())


In [ ]:
# Generate embeddings for descriptions and store them in the embeddings table
model = SentenceTransformer(EMB_MODEL_NAME)
tx_df = con.execute("SELECT tx_id, description FROM transactions ORDER BY tx_id").fetchdf()
texts = tx_df['description'].astype(str).tolist()
embs = model.encode(texts, convert_to_numpy=True)

# Persist embeddings as JSON strings for PoC
emb_job_id = hashlib.sha1(str(time.time()).encode()).hexdigest()[:10]
rows = []
for tx_id, emb in zip(tx_df['tx_id'].tolist(), embs):
    rows.append((tx_id, json.dumps(emb.tolist()), EMB_MODEL_NAME, emb_job_id))

emb_df = pd.DataFrame(rows, columns=['tx_id','emb_json','emb_model','emb_job_id'])
con.execute('INSERT INTO embeddings SELECT * FROM emb_df', {'emb_df': emb_df})
print('Stored', len(rows), 'embeddings in embeddings table')


In [ ]:
# Build a nearest-neighbour index (scikit-learn) in memory for retrieval
emb_matrix = np.vstack([np.array(json.loads(x), dtype=np.float32) for x in con.execute('SELECT emb_json FROM embeddings ORDER BY tx_id').fetchdf()['emb_json']])
tx_ids = con.execute('SELECT tx_id FROM embeddings ORDER BY tx_id').fetchdf()['tx_id'].tolist()

nn = NearestNeighbors(n_neighbors=TOP_K, metric='cosine', algorithm='brute')
nn.fit(emb_matrix)
print('NearestNeighbors index built on', emb_matrix.shape[0], 'vectors')


In [ ]:
# Define LLM caller (mock if no real LLM key)
def call_llm(prompt: str, model_name: str = 'mock-llm', top_k_retrieved=None):
    """
    Call an LLM provider if available, otherwise return a deterministic mock response.
    Returns: dict with keys: text, parsed (dict or None), usage (dict or None), latency_ms (int)
    """
    start = time.time()
    if REAL_LLM_AVAILABLE and openai:
        # Use OpenAI chat completion (gpt-3.5-turbo) as example
        try:
            resp = openai.ChatCompletion.create(
                model='gpt-3.5-turbo',
                messages=[{"role": "system", "content": "You are a fraud-analyst assistant."}, {"role": "user", "content": prompt}],
                temperature=0.0,
                max_tokens=400
            )
            text = resp['choices'][0]['message']['content']
            usage = resp.get('usage', None)
            latency_ms = int((time.time() - start) * 1000)
            # Try to parse JSON from the response
            parsed = None
            try:
                parsed = json.loads(text.strip())
            except Exception:
                # attempt to extract JSON substring
                import re
                m = re.search(r"(\{.*\})", text, re.DOTALL)
                if m:
                    try:
                        parsed = json.loads(m.group(1))
                    except Exception:
                        parsed = None
            return {"text": text, "parsed": parsed, "usage": usage, "latency_ms": latency_ms}
        except Exception as e:
            # fallback to mock if API call fails
            print('LLM call failed, using mock. Error:', e)

    # Mock deterministic response: derive risk_score from presence of labeled fraud in retrieved
    retrieved = top_k_retrieved or []
    # simple heuristic: if any retrieved label == 1 (we consider tx1002 and tx1005 as fraud-like in synthetic data)
    fraud_like_ids = set(['tx1002','tx1005'])
    evidence = [tid for tid in retrieved if tid in fraud_like_ids]
    base_score = 0.2
    if evidence:
        base_score += 0.6
    # increase score for large amount keyword
    if 'transfer' in prompt.lower() or 'large' in prompt.lower():
        base_score = min(0.95, base_score + 0.15)
    parsed = {"risk_score": round(base_score, 2), "explanation": f"Found similar fraud-like cases: {evidence}", "evidence": evidence}
    latency_ms = int((time.time() - start) * 1000)
    return {"text": json.dumps(parsed), "parsed": parsed, "usage": None, "latency_ms": latency_ms}

print('LLM caller defined (REAL_LLM_AVAILABLE=', REAL_LLM_AVAILABLE, ')')


In [ ]:
# Simulate an incoming transaction that needs fraud evaluation
incoming = {
    'tx_id': 'tx_live_9001',
    'account_id': 'acct1',
    'amount': 8200.0,
    'currency': 'USD',
    'merchant': 'MerchantZ',
    'merchant_mcc': '6011',
    'timestamp': '2026-01-07 14:23:00',
    'description': 'Immediate large transfer to new beneficiary account',
    'device_info': 'mobile',
    'geo_country': 'CN'
}

# Insert incoming transaction
con.execute("INSERT INTO transactions (tx_id, account_id, amount, currency, merchant, merchant_mcc, timestamp, description, device_info, geo_country, ingestion_job_id, raw_source, pii_masked) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
            (incoming['tx_id'], incoming['account_id'], incoming['amount'], incoming['currency'], incoming['merchant'], incoming['merchant_mcc'], incoming['timestamp'], incoming['description'], incoming['device_info'], incoming['geo_country'], ingestion_job_id, 'realtime', False))
print('Inserted incoming transaction', incoming['tx_id'])

# Compute embedding for incoming description
q_emb = model.encode([incoming['description']])[0].astype(np.float32)

# Retrieve top-K similar historical transactions
distances, indices = nn.kneighbors([q_emb], n_neighbors=TOP_K, return_distance=True)
retrieved_indices = indices[0].tolist()
retrieved_tx_ids = [tx_ids[i] for i in retrieved_indices]
print('Retrieved tx_ids:', retrieved_tx_ids)

# Get retrieved case details
placeholders = ','.join([f"'{tid}'" for tid in retrieved_tx_ids]) if retrieved_tx_ids else "''"
retrieved_df = con.execute(f"SELECT tx_id, account_id, amount, merchant, description FROM transactions WHERE tx_id IN ({placeholders}) ORDER BY tx_id").fetchdf()
print('Retrieved details:\n', retrieved_df)

# Build prompt for RAG-style LLM call
prompt = f"""
You are a fraud analyst assistant. Given an incoming transaction and historical similar transactions, provide:\n
1) A risk score from 0.0 to 1.0 (1.0 = highest risk).\n
2) A short explanation (1-3 sentences) citing the retrieved transaction IDs and features that indicate fraud.\n
3) A JSON output with keys: risk_score, explanation, evidence (list of tx_ids).\n
Incoming transaction:\n{json.dumps(incoming, indent=2)}\n
Retrieved historical cases:\n{retrieved_df.to_json(orient='records')}\n
Return only valid JSON (object) as the top-level content.
"""

# Call LLM (real or mock)
llm_start = time.time()
llm_resp = call_llm(prompt, model_name='gpt-mock-or-api', top_k_retrieved=retrieved_tx_ids)
llm_latency = llm_resp['latency_ms']
llm_text = llm_resp['text']
parsed = llm_resp['parsed']
usage = llm_resp['usage']

print('LLM response (parsed):', parsed)


In [ ]:
# Persist LLM result with provenance into DuckDB
import uuid
res_id = str(uuid.uuid4())
prompt_hash = hashlib.sha256(prompt.encode()).hexdigest()
risk_score = None
if parsed and isinstance(parsed, dict):
    risk_score = float(parsed.get('risk_score', None)) if parsed.get('risk_score', None) is not None else None

con.execute(
    "INSERT INTO llm_results (id, tx_id, llm_model, llm_provider, llm_prompt_hash, llm_prompt, llm_response, parsed_response, risk_score, evidence_tx_ids, call_latency_ms, usage) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
    (
        res_id,
        incoming['tx_id'],
        'gpt-mock-or-api',
        'mock' if not REAL_LLM_AVAILABLE else 'openai',
        prompt_hash,
        prompt if REAL_LLM_AVAILABLE else None,  # store prompt only if allowed/real
        llm_text,
        json.dumps(parsed) if parsed is not None else None,
        risk_score,
        json.dumps(retrieved_tx_ids),
        llm_latency,
        json.dumps(usage) if usage is not None else None
    )
)

print('Stored llm_result id=', res_id)


In [ ]:
# Create a simple enriched view (transactions + latest llm_result)
con.execute('''
CREATE OR REPLACE VIEW tx_enriched AS
SELECT t.*, l.llm_model, l.llm_provider, l.llm_response, l.parsed_response, l.risk_score, l.evidence_tx_ids, l.created_at AS llm_ts
FROM transactions t
LEFT JOIN (
  SELECT id, tx_id, llm_model, llm_provider, llm_response, parsed_response, risk_score, evidence_tx_ids, created_at
  FROM llm_results
  QUALIFY ROW_NUMBER() OVER (PARTITION BY tx_id ORDER BY created_at DESC) = 1
) l USING (tx_id);
''')

print('Enriched view created. Sample high-risk items:')
print(con.execute("SELECT tx_id, amount, description, risk_score FROM tx_enriched ORDER BY risk_score DESC NULLS LAST LIMIT 10").fetchdf())


## What you can do next

- Replace the mock LLM with an on-prem or external LLM by setting OPENAI_API_KEY (or adding another provider) and adjusting the call_llm function.
- For production scale, store embeddings in a dedicated vector store (FAISS/Milvus/Pinecone) and keep only references in DuckDB.
- Add PII masking logic before any external API calls, and store prompt hashes if prompts must not be persisted.
- Add data-quality checks, lineage writes, and a model registry for full governance.

This notebook shows the core data-management pattern: treat LLM outputs as first-class data (store responses, parsed fields, provenance, and link them back to source records).